In [1]:
from ipywidgets import interact
from matplotlib import pyplot as plt
import time

In [2]:
from sequence.kernel.timeline import Timeline
from sequence.topology.node import QuantumRepeater, MiddleNode
from sequence.components.optical_channel import QuantumChannel, ClassicalChannel

In [3]:
class Parent():
    def __init__(self, timeline):
        self.timeline = timeline
        self.pop_times = []
        
    def pop(self, **kwargs):
        self.pop_times.append(self.timeline.now() * 1e-12)

In [4]:
def test(sim_time, cc_delay, qc_atten, qc_dist):
    """
    sim_time: duration of simulation time (ms)
    cc_delay: delay on classical channels (ns)
    qc_atten: attenuation on quantum channels (db/m)
    qc_dist: distance of quantum channels (km)
    """
    cc_delay *= 1e6
    qc_dist *= 1e3
    tl = Timeline(sim_time * 1e9)
    
    r1 = QuantumRepeater("r1", tl)
    r2 = QuantumRepeater("r2", tl)
    m1 = MiddleNode("m1", tl)
    
    cc1 = ClassicalChannel("cc_r1_m1", tl, 2e-4, 1e3, delay=cc_delay)
    cc1.set_ends(r1, m1)
    cc2 = ClassicalChannel("cc_r2_m1", tl, 2e-4, 1e3, delay=cc_delay)
    cc2.set_ends(r2, m1)
    cc3 = ClassicalChannel("cc_r1_m2", tl, 2e-4, 1e3, delay=cc_delay)
    cc3.set_ends(r1, r2)
    
    qc1 = QuantumChannel("qc_r1_m1", tl, qc_atten, qc_dist)
    qc1.set_ends(r1, m1)
    qc2 = QuantumChannel("qc_r2_m1", tl, qc_atten, qc_dist)
    qc2.set_ends(r2, m1)
    
    r1.eg_add_middle(m1)
    r2.eg_add_middle(m1)
    r1.eg_add_others(r2)
    r2.eg_add_others(r1)
    m1.eg_add_others(r1)
    m1.eg_add_others(r2)
    
    # to keep track of entangled pairs
    parent = Parent(tl)
    r1.eg.upper_protocols.append(parent)
        
    tl.init()
    r1.load_events()
    
    tick = time.time()
    tl.run()
    print("execution time %.2f sec" % (time.time() - tick))
    
    data = parent.pop_times
    plt.plot(data, range(1, len(data) + 1), marker="o")
    plt.xlabel("Simulation Time (s)")
    plt.ylabel("Aggregated Number of Entangled Memory")
    plt.show()
    

In [5]:
interactive_plot = interact(test, sim_time=(100, 1000, 100), cc_delay=(100, 10000, 100), qc_atten=[1e-5, 2e-5, 3e-5], qc_dist=(1, 10, 1))
interactive_plot

interactive(children=(IntSlider(value=500, description='sim_time', max=1000, min=100, step=100), IntSlider(val…

<function __main__.test(sim_time, cc_delay, qc_atten, qc_dist)>